In [1]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.parser import *
import json
import requests
from pprint import pprint
from config import driver, username, password, host, port, database
from sqlalchemy import create_engine
from time import ctime

In [2]:
pd.options.display.max_columns = 30

In [3]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect()

In [4]:
contentChoiceDF = pd.read_sql_table('contentChoice', connection)
contentChoiceDF

,id,locationName,shortName,city,latitude,longitude,homePage
0,1,"University of California, Los Angeles",UCLA,Los Angeles,34.079560,-118.444940,Y
1,2,"University of California, Berkeley",UCB,Berkeley,37.871853,-122.258423,N
2,3,"University of California, San Diego",UCSD,La Jolla,32.878880,-117.235930,N
3,4,"University of California, Santa Barbara",UCSB,Santa Barbara,34.413963,-119.848946,N
4,5,"University of California, Irvine",UCI,Irvine,33.640990,-117.844370,N


In [5]:
locationNames = []
latitudes = []
longitudes = []

for row in contentChoiceDF:
    print(row)

id
locationName
shortName
city
latitude
longitude
homePage


In [6]:
city = "Los Angeles"
lat = 34.0522
lon = -118.2437

In [7]:
## Get initial endpoint for city
# Assign initial api url to a variable
lat_lon_url = f"https://api.weather.gov/points/{lat},{lon}"

# API call to retrieve enpoints for location of lat and lon
response = requests.get(lat_lon_url).json()

In [8]:
pprint(response)

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
              {'@version': '1.1',
               '@vocab': 'https://api.weather.gov/ontology#',
               'bearing': {'@type': 's:QuantitativeValue'},
               'city': 's:addressLocality',
               'county': {'@type': '@id'},
               'distance': {'@id': 's:Distance',
                            '@type': 's:QuantitativeValue'},
               'forecastGridData': {'@type': '@id'},
               'forecastOffice': {'@type': '@id'},
               'geo': 'http://www.opengis.net/ont/geosparql#',
               'geometry': {'@id': 's:GeoCoordinates',
                            '@type': 'geo:wktLiteral'},
               'publicZone': {'@type': '@id'},
               's': 'https://schema.org/',
               'state': 's:addressRegion',
               'unit': 'http://codes.wmo.int/common/unit/',
               'unitCode': {'@id': 's:unitCode', '@type': '@id'},
               'value': {'@id': 's:valu

In [9]:
# Specify the URL
# Major cities in California
weekly_url = response["properties"]["forecast"]
hourly_url = response["properties"]["forecastHourly"]
tz =  response["properties"]["timeZone"]

In [10]:
# Make request and store response
weekly_response = requests.get(weekly_url).json()
weekly_response

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-118.2618399, 34.062663],
    [-118.25719459999999, 34.0404501],
    [-118.23047399999999, 34.0442838],
    [-118.23511409999999, 34.0664973],
    [-118.2618399, 34.062663]]]},
 'properties': {'updated': '2021-11-01T21:24:44+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2021-11-01T21:41:55+00:00',
  'updateTime': '2021-11-01T21:24:44+00:00',
  'validTimes': '2021-11-01T15:00:00+00:00/P7DT10H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 114.9096},
  'periods': [{'number': 1,
    'name': 'This Afternoon',
    'startTime': '2021-11-01T14:00:00-07:00',
    'endTime': '2021-11-01

In [11]:
# # Find generatedAt string; convert to datetime object
# call_datetime_str = weekly_response["properties"]["generatedAt"]
# call_datetime = parse(call_datetime_str)

In [12]:
# Find period forecasts; assign to variable
period_forecasts = weekly_response["properties"]["periods"]

In [13]:
weekly_forecast = []
for i, period_forecast in enumerate(period_forecasts):
    # Isolate date and time from StartTime and EndTime; convert to datetime object
    start_date = datetime.strptime(period_forecasts[i]["startTime"].split("T")[0],"%Y-%m-%d").date()
    start_time = datetime.strptime(period_forecasts[i]["startTime"].split("T")[1],"%H:%M:%S%z").time()
    end_date = datetime.strptime(period_forecasts[i]["endTime"].split("T")[0],"%Y-%m-%d").date()
    end_time = datetime.strptime(period_forecasts[i]["endTime"].split("T")[1],"%H:%M:%S%z").time()
    
    # Get min, max wind speeds; get wind speed units
    wind_speed = period_forecasts[i]["windSpeed"].split(" ")
    if len(wind_speed) == 2:
        min_wind_speed = float(wind_speed[0])
        max_wind_speed = np.nan
        wind_speed_unit = wind_speed[1]
    elif len(wind_speed) == 4:
        min_wind_speed = float(wind_speed[0])
        max_wind_speed = float(wind_speed[2])
        wind_speed_unit = wind_speed[3]

    # Append dates and times to period_forecast dictionary
    period_forecast["city"] = city
    period_forecast["startDate"] = start_date
    period_forecast["start_time"] = start_time
    period_forecast["endDate"] = end_date
    period_forecast["end_time"] = end_time
    period_forecast["minWindSpeed"] = min_wind_speed
    period_forecast["maxWindSpeed"] = max_wind_speed
    period_forecast["windSpeedUnit"] = wind_speed_unit
    period_forecast["latitude"] = lat
    period_forecast["longitude"] = lon
    period_forecast["retrievalDateTime"] = ctime()
    
    # Append period_forcast to weekly_forecast list
    weekly_forecast.append(period_forecast)

# Create dataframe from "weekly_forecast" list
weekly_forecast_raw_df = pd.DataFrame(weekly_forecast)

In [14]:
# Copy weekly_forecast_raw_df to working dataframe
weekly_forecast_working_df = weekly_forecast_raw_df

# Convert startTime from string to datetime object
weekly_forecast_working_df["startTime"] = pd.to_datetime(weekly_forecast_working_df["startTime"])

# Convert endtTime from string to datetime object
weekly_forecast_working_df["endTime"] = pd.to_datetime(weekly_forecast_working_df["endTime"])

# Convert isDayTime from string to boolean
weekly_forecast_working_df.replace({"isDaytime": {"True": True, "False": False}}, inplace=True)

# Reasign column names
weekly_forecast_working_df.rename(columns={"startTime":"startDateTime", "endTime":"endDateTime", "start_time":"startTime", "end_time":"endTime", "number":"responseNumber", "name":"responseName"}, inplace=True)

# Rearange columns of dataframe
weekly_forecast_working_df = weekly_forecast_working_df[['responseNumber', 'responseName', 'city', 'latitude', 'longitude','startDateTime', 'startDate', 'startTime', 'endDateTime', 'endDate', 'endTime', 'isDaytime', 'temperature', 'temperatureUnit', 'temperatureTrend', 'windSpeed','minWindSpeed', 'maxWindSpeed', 'windSpeedUnit', 'windDirection', 'icon', 'shortForecast', 'detailedForecast', 'retrievalDateTime']]

In [15]:
# Copy working dataframe to final dataframe
weekly_forecast_df = weekly_forecast_working_df

In [16]:
# Make hourly forecast request and store response
hourly_response = requests.get(hourly_url).json()

In [17]:
# # Find generatedAt string; convert to datetime object
# request_datetime_str = hourly_response["properties"]["generatedAt"]
# request_datetime = parse(call_datetime_str)

In [18]:
# Find hour forecasts; assign to variable
hour_forecasts = hourly_response["properties"]["periods"]
hour_forecasts

[{'number': 1,
  'name': '',
  'startTime': '2021-11-01T14:00:00-07:00',
  'endTime': '2021-11-01T15:00:00-07:00',
  'isDaytime': True,
  'temperature': 70,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'windSpeed': '5 mph',
  'windDirection': 'WNW',
  'icon': 'https://api.weather.gov/icons/land/day/bkn?size=small',
  'shortForecast': 'Mostly Cloudy',
  'detailedForecast': ''},
 {'number': 2,
  'name': '',
  'startTime': '2021-11-01T15:00:00-07:00',
  'endTime': '2021-11-01T16:00:00-07:00',
  'isDaytime': True,
  'temperature': 70,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'windSpeed': '5 mph',
  'windDirection': 'WNW',
  'icon': 'https://api.weather.gov/icons/land/day/bkn?size=small',
  'shortForecast': 'Mostly Cloudy',
  'detailedForecast': ''},
 {'number': 3,
  'name': '',
  'startTime': '2021-11-01T16:00:00-07:00',
  'endTime': '2021-11-01T17:00:00-07:00',
  'isDaytime': True,
  'temperature': 68,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'wind

In [19]:
hourly_forecast = []
for i, hour_forecast in enumerate(hour_forecasts):
    # Isolate date and time from StartTime and EndTime; convert to datetime object
    start_date = datetime.strptime(hour_forecasts[i]["startTime"].split("T")[0],"%Y-%m-%d").date()
    start_time = datetime.strptime(hour_forecasts[i]["startTime"].split("T")[1],"%H:%M:%S%z").time()
    end_date = datetime.strptime(hour_forecasts[i]["endTime"].split("T")[0],"%Y-%m-%d").date()
    end_time = datetime.strptime(hour_forecasts[i]["endTime"].split("T")[1],"%H:%M:%S%z").time()
    
    # Get hour wind speed; get wind speed units
    wind_speed = hour_forecasts[i]["windSpeed"].split(" ")
    hour_wind_speed = float(wind_speed[0])
    wind_speed_unit = wind_speed[1]

    # Append dates and times to hour_forecast dictionary
    hour_forecast["city"] = city
    hour_forecast["startDate"] = start_date
    hour_forecast["start_time"] = start_time
    hour_forecast["endDate"] = end_date
    hour_forecast["end_time"] = end_time
    hour_forecast["hourWindSpeed"] = hour_wind_speed
    hour_forecast["windSpeedUnit"] = wind_speed_unit
    hour_forecast["latitude"] = lat
    hour_forecast["longitude"] = lon
    hour_forecast["retrievalDateTime"] = ctime()
    
    # Append period_forcast to hourly_forecast list
    hourly_forecast.append(hour_forecast)

# Create dataframe from "hourly_forecast" list
hourly_forecast_raw_df = pd.DataFrame(hourly_forecast)

In [20]:
# Copy hourly_forecast_raw_df to working dataframe
hourly_forecast_working_df = hourly_forecast_raw_df

# Convert startTime from string to datetime object
hourly_forecast_working_df["startTime"] = pd.to_datetime(hourly_forecast_working_df["startTime"])

# Convert endtTime from string to datetime object
hourly_forecast_working_df["endTime"] = pd.to_datetime(hourly_forecast_working_df["endTime"])

# Convert isDayTime from string to boolean
hourly_forecast_working_df.replace({"isDaytime": {"True": True, "False": False}}, inplace=True)

# Reasign column names
hourly_forecast_working_df.rename(columns={"startTime":"startDateTime", "endTime":"endDateTime", "start_time":"startTime", "end_time":"endTime", "number":"responseNumber"}, inplace=True)

# Drop empty columns "name" and "detailedForecast"
hourly_forecast_working_df.drop(columns=["name", "detailedForecast"], inplace=True)

# Rearange columns of dataframe
hourly_forecast_working_df = hourly_forecast_working_df[['responseNumber', 'city', 'latitude', 'longitude','startDateTime', 'startDate', 'startTime', 'endDateTime', 'endDate', 'endTime', 'isDaytime', 'temperature', 'temperatureUnit', 'temperatureTrend', 'windSpeed','hourWindSpeed', 'windSpeedUnit', 'windDirection', 'icon', 'shortForecast', 'retrievalDateTime']]

In [21]:
# Copy working dataframe to final dataframe
hourly_forecast_df = hourly_forecast_working_df

In [22]:
hourly_forecast_df

,responseNumber,city,latitude,longitude,startDateTime,startDate,startTime,endDateTime,endDate,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,hourWindSpeed,windSpeedUnit,windDirection,icon,shortForecast,retrievalDateTime
0,1,Los Angeles,34.0522,-118.2437,2021-11-01 14:00:00-07:00,2021-11-01,14:00:00,2021-11-01 15:00:00-07:00,2021-11-01,15:00:00,True,70,F,None,5 mph,5.0,mph,WNW,https://api.weather.gov/icons/land/day/bkn?siz...,Mostly Cloudy,Mon Nov 1 15:52:20 2021
1,2,Los Angeles,34.0522,-118.2437,2021-11-01 15:00:00-07:00,2021-11-01,15:00:00,2021-11-01 16:00:00-07:00,2021-11-01,16:00:00,True,70,F,None,5 mph,5.0,mph,WNW,https://api.weather.gov/icons/land/day/bkn?siz...,Mostly Cloudy,Mon Nov 1 15:52:20 2021
2,3,Los Angeles,34.0522,-118.2437,2021-11-01 16:00:00-07:00,2021-11-01,16:00:00,2021-11-01 17:00:00-07:00,2021-11-01,17:00:00,True,68,F,None,5 mph,5.0,mph,WNW,https://api.weather.gov/icons/land/day/bkn?siz...,Mostly Cloudy,Mon Nov 1 15:52:20 2021
3,4,Los Angeles,34.0522,-118.2437,2021-11-01 17:00:00-07:00,2021-11-01,17:00:00,2021-11-01 18:00:00-07:00,2021-11-01,18:00:00,True,66,F,None,5 mph,5.0,mph,WNW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,Mon Nov 1 15:52:20 2021
4,5,Los Angeles,34.0522,-118.2437,2021-11-01 18:00:00-07:00,2021-11-01,18:00:00,2021-11-01 19:00:00-07:00,2021-11-01,19:00:00,False,63,F,None,5 mph,5.0,mph,WNW,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,Mon Nov 1 15:52:20 2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,152,Los Angeles,34.0522,-118.2437,2021-11-07 20:00:00-08:00,2021-11-07,20:00:00,2021-11-07 21:00:00-08:00,2021-11-07,21:00:00,False,58,F,None,5 mph,5.0,mph,SW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,Mon Nov 1 15:52:20 2021
152,153,Los Angeles,34.0522,-118.2437,2021-11-07 21:00:00-08:00,2021-11-07,21:00:00,2021-11-07 22:00:00-08:00,2021-11-07,22:00:00,False,57,F,None,5 mph,5.0,mph,SW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,Mon Nov 1 15:52:20 2021
153,154,Los Angeles,34.0522,-118.2437,2021-11-07 22:00:00-08:00,2021-11-07,22:00:00,2021-11-07 23:00:00-08:00,2021-11-07,23:00:00,False,56,F,None,5 mph,5.0,mph,E,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,Mon Nov 1 15:52:20 2021
154,155,Los Angeles,34.0522,-118.2437,2021-11-07 23:00:00-08:00,2021-11-07,23:00:00,2021-11-08 00:00:00-08:00,2021-11-08,00:00:00,False,56,F,None,5 mph,5.0,mph,E,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,Mon Nov 1 15:52:20 2021


In [23]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)
connection = engine.connect().execution_options(autocommit = True)

In [24]:
weekly_forecast_df

,responseNumber,responseName,city,latitude,longitude,startDateTime,startDate,startTime,endDateTime,endDate,endTime,isDaytime,temperature,temperatureUnit,temperatureTrend,windSpeed,minWindSpeed,maxWindSpeed,windSpeedUnit,windDirection,icon,shortForecast,detailedForecast,retrievalDateTime
0,1,This Afternoon,Los Angeles,34.0522,-118.2437,2021-11-01 14:00:00-07:00,2021-11-01,14:00:00,2021-11-01 18:00:00-07:00,2021-11-01,18:00:00,True,70,F,None,5 mph,5.0,NaN,mph,WNW,https://api.weather.gov/icons/land/day/bkn?siz...,Partly Sunny,"Partly sunny, with a high near 70. West northw...",Mon Nov 1 15:52:18 2021
1,2,Tonight,Los Angeles,34.0522,-118.2437,2021-11-01 18:00:00-07:00,2021-11-01,18:00:00,2021-11-02 06:00:00-07:00,2021-11-02,06:00:00,False,56,F,None,0 to 5 mph,0.0,5.0,mph,ENE,https://api.weather.gov/icons/land/night/bkn/f...,Mostly Cloudy then Patchy Fog,"Patchy fog after 5am. Mostly cloudy, with a lo...",Mon Nov 1 15:52:18 2021
2,3,Tuesday,Los Angeles,34.0522,-118.2437,2021-11-02 06:00:00-07:00,2021-11-02,06:00:00,2021-11-02 18:00:00-07:00,2021-11-02,18:00:00,True,72,F,None,0 to 5 mph,0.0,5.0,mph,SE,https://api.weather.gov/icons/land/day/fog/bkn...,Patchy Fog then Partly Sunny,"Patchy fog before 11am. Partly sunny, with a h...",Mon Nov 1 15:52:18 2021
3,4,Tuesday Night,Los Angeles,34.0522,-118.2437,2021-11-02 18:00:00-07:00,2021-11-02,18:00:00,2021-11-03 06:00:00-07:00,2021-11-03,06:00:00,False,57,F,None,0 to 5 mph,0.0,5.0,mph,SE,https://api.weather.gov/icons/land/night/sct?s...,Partly Cloudy,"Partly cloudy, with a low around 57. Southeast...",Mon Nov 1 15:52:18 2021
4,5,Wednesday,Los Angeles,34.0522,-118.2437,2021-11-03 06:00:00-07:00,2021-11-03,06:00:00,2021-11-03 18:00:00-07:00,2021-11-03,18:00:00,True,75,F,None,0 to 5 mph,0.0,5.0,mph,NW,https://api.weather.gov/icons/land/day/sct?siz...,Mostly Sunny,"Mostly sunny, with a high near 75. Northwest w...",Mon Nov 1 15:52:18 2021
5,6,Wednesday Night,Los Angeles,34.0522,-118.2437,2021-11-03 18:00:00-07:00,2021-11-03,18:00:00,2021-11-04 06:00:00-07:00,2021-11-04,06:00:00,False,57,F,None,0 to 5 mph,0.0,5.0,mph,NNW,https://api.weather.gov/icons/land/night/fog?s...,Patchy Fog,"Patchy fog after 11pm. Mostly cloudy, with a l...",Mon Nov 1 15:52:18 2021
6,7,Thursday,Los Angeles,34.0522,-118.2437,2021-11-04 06:00:00-07:00,2021-11-04,06:00:00,2021-11-04 18:00:00-07:00,2021-11-04,18:00:00,True,77,F,None,0 to 10 mph,0.0,10.0,mph,W,https://api.weather.gov/icons/land/day/fog/sct...,Patchy Fog then Mostly Sunny,"Patchy fog before 11am. Mostly sunny, with a h...",Mon Nov 1 15:52:18 2021
7,8,Thursday Night,Los Angeles,34.0522,-118.2437,2021-11-04 18:00:00-07:00,2021-11-04,18:00:00,2021-11-05 06:00:00-07:00,2021-11-05,06:00:00,False,55,F,None,5 to 10 mph,5.0,10.0,mph,NNW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 55.",Mon Nov 1 15:52:18 2021
8,9,Friday,Los Angeles,34.0522,-118.2437,2021-11-05 06:00:00-07:00,2021-11-05,06:00:00,2021-11-05 18:00:00-07:00,2021-11-05,18:00:00,True,78,F,None,5 to 10 mph,5.0,10.0,mph,NNW,https://api.weather.gov/icons/land/day/few?siz...,Sunny,"Sunny, with a high near 78.",Mon Nov 1 15:52:18 2021
9,10,Friday Night,Los Angeles,34.0522,-118.2437,2021-11-05 18:00:00-07:00,2021-11-05,18:00:00,2021-11-06 06:00:00-07:00,2021-11-06,06:00:00,False,56,F,None,5 to 10 mph,5.0,10.0,mph,NNW,https://api.weather.gov/icons/land/night/few?s...,Mostly Clear,"Mostly clear, with a low around 56.",Mon Nov 1 15:52:18 2021


In [ ]:
session.close()

In [25]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [44]:
connection_string = f"{driver}://{username}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string)

In [28]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [29]:
Base.classes.keys()

['contentChoice',
 'histDailyForecastTB',
 'histHourlyForecastTB',
 'dailyForecastTB',
 'hourlyForecastTB']

In [37]:
# Save reference to the tables
ContentChoice = Base.classes.contentChoice
DailyForecastTB = Base.classes.dailyForecastTB
HourlyForecastTB = Base.classes.hourlyForecastTB

In [41]:
session = Session(engine)
session.query(DailyForecastTB).delete()
session.query(HourlyForecastTB).delete()

0

In [ ]:
weekly_forecast_df.to_sql('dailyForecastTB',connection, if_exists='replace', index=False)

In [ ]:
hourly_forecast_df.to_sql('hourlyForecastTB',connection, if_exists='append', index=False)

In [ ]:
DailyForecastTblDF = pd.read_sql_table('dailyForecastTB', connection)
DailyForecastTblDF

In [ ]:
HourlyForecastTblDF = pd.read_sql_table('hourlyForecastTB', connection)
HourlyForecastTblDF